In [ ]:
# File: fitting.ipynb
# Author: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [9]:
import os
import random

from frontend import App

app = App.create("fitting")

codim_ipc_root = "/tmp/Codim-IPC"
checkout_list = [
    # "Projects/FEMShell/input/dress_knife",
    "Projects/FEMShell/input/multilayer",
    "Projects/FEMShell/input/Rumba_Dancing",
]
app.extra.sparse_clone(
    "https://github.com/ipc-sim/Codim-IPC", codim_ipc_root, checkout_list
)

stage_path = os.path.join(codim_ipc_root, checkout_list[0], "stage.obj")
body_path = os.path.join(codim_ipc_root, checkout_list[1], "shell0.obj")

V, F, S = app.extra.load_CIPC_stitch_mesh(stage_path)
app.asset.add.tri("dress", V, F)
app.asset.add.stitch("glue", S)

V, F = app.mesh.load_tri(body_path)
app.asset.add.tri("body", V, F)

scene = app.scene.create()
scene.add("dress").stitch("glue").rotate(-90, "x")

jitter = 0.01 * random.random()
body = scene.add("body").at(0, -0.78 + jitter, 0).pin()
body_dir = os.path.join(codim_ipc_root, checkout_list[1])
frame = 1
while True:
    path = os.path.join(body_dir, f"shell{frame}.obj")
    if os.path.exists(path):
        V, _ = app.mesh.load_tri(path)
        body.move_to(V, 0.5 + 0.025 * frame)
        frame += 1
    else:
        break

fixed = scene.build().report()
fixed.preview(options={"pin": False})

build: 100%|██████████████████████████| 10/10 [00:00<00:00, 45.54it/s]


#vert,#tri,#pin,#stitch_ind
"43,275","84,159","12,811",791


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8900443613529205), quaterni…

interactive(children=(FloatSlider(value=0.0, description='time', max=3.5, step=0.01), Output()), _dom_classes=…

In [90]:
param = (
    app.session.param()
    .set("fitting")
    .set("dt", 0.0004)
    .set("frames", 240)
    .set("friction", 0.0)
    .set("model-shell", "arap")
    .set("contact-ghat", 0.4e-3) # 0.5e3  parameter controls the activation threshold of contact. Lower values increase contact density:
    .set("strain-limit-eps", 0.06) #0.1 Make it smaller to strongly limit how much the cloth can stretch
    .set("cg-tol", 1e-6)
    .set("cg-max-iter",1000000)
    .set("bend",17)
)
param.dyn("fitting").time(0.15).hold().change(False)
param.dyn("dt").time(0.15).hold().change(0.0338)   #since 0.032 always converges but 0.034 fails 

session = app.session.create(fixed)
session.start(param).preview()
session.stream()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(position=(0.0, 0.0, 1.8900443613529205), quaterni…

HTML(value='<table class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n   …

HTML(value='')

In [91]:
from collections import defaultdict

# Group PCG iterations by simulation time
pcg_iters = session.get.log.numbers("pcg-iter")
frame_map = defaultdict(list)

for sim_time, count in pcg_iters:
    frame_map[sim_time].append(count)

# Print average PCG iterations per simulation time step
for time, counts in sorted(frame_map.items()):
    avg_iters = sum(counts) / len(counts)
    print(f"Time {time:.6f}: {avg_iters:.1f} average PCG iterations over {len(counts)} Newton steps")


Time 0.000000: 230.7 average PCG iterations over 6 Newton steps
Time 0.000104: 251.6 average PCG iterations over 5 Newton steps
Time 0.000213: 270.4 average PCG iterations over 5 Newton steps
Time 0.000315: 296.8 average PCG iterations over 6 Newton steps
Time 0.000436: 307.3 average PCG iterations over 6 Newton steps
Time 0.000554: 315.5 average PCG iterations over 6 Newton steps
Time 0.000676: 320.0 average PCG iterations over 6 Newton steps
Time 0.000794: 321.9 average PCG iterations over 7 Newton steps
Time 0.000899: 332.9 average PCG iterations over 8 Newton steps
Time 0.001012: 318.1 average PCG iterations over 7 Newton steps
Time 0.001128: 303.4 average PCG iterations over 7 Newton steps
Time 0.001233: 301.3 average PCG iterations over 6 Newton steps
Time 0.001351: 297.4 average PCG iterations over 7 Newton steps
Time 0.001466: 292.5 average PCG iterations over 8 Newton steps
Time 0.001575: 290.6 average PCG iterations over 8 Newton steps
Time 0.001686: 263.4 average PCG iterati

In [92]:
import os
import csv
import json
from datetime import datetime

# Unique run name (can be changed to use a config label or run index)
run_id = datetime.now().strftime("run_%Y%m%d_%H%M%S")  # e.g., run_20250504_173212

# Base folder and subfolder for this run
log_base = "logs"
log_run_folder = os.path.join(log_base, run_id)
os.makedirs(log_run_folder, exist_ok=True)

# Collect and save logs
log_names = session.get.log.names()
log_data = {}

for name in log_names:
    try:
        entries = session.get.log.numbers(name)
        log_data[name] = entries

        # Save each log to CSV
        csv_path = os.path.join(log_run_folder, f"{name}.csv")
        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["time_or_frame", "value"])
            writer.writerows(entries)

    except Exception as e:
        print(f"Could not log {name}: {e}")

# Save as JSON as well
json_path = os.path.join(log_run_folder, "logs_all.json")
with open(json_path, "w") as f:
    json.dump(log_data, f, indent=2)

print(f"✔ Logs for this run saved in: {log_run_folder}")


✔ Logs for this run saved in: logs/run_20250505_052012


In [93]:
import shutil

# Define folder and zip filename
folder_to_zip = "logs"
zip_filename = "logs_archive"

# Create a ZIP file (will create logs_archive.zip)
shutil.make_archive(zip_filename, 'zip', folder_to_zip)

print(f"✔ Zipped folder saved as: {zip_filename}.zip")


✔ Zipped folder saved as: logs_archive.zip


In [27]:
# get a list of log names
logs = session.get.log.names()
assert "time-per-frame" in logs
assert "newton-steps" in logs
assert "pcg-iter" in logs

# get a list of time per video frame
msec_per_video = session.get.log.numbers("time-per-frame")
frame_to_time = session.get.log.numbers("frame-to-time")

# compute the average time per video frame
print("avg per frame:", sum([n for _, n in msec_per_video]) / len(msec_per_video))

# get a list of newton steps
newton_steps = session.get.log.numbers("newton-steps")
pcg_iterations = session.get.log.numbers("pcg-iter")

# compute the average of consumed newton steps
print("avg newton steps:", sum([n for _, n in newton_steps]) / len(newton_steps))
print("avg pcg iterations per newton step :", sum([n for _, n in pcg_iterations]) / len(pcg_iterations))

avg per frame: 1387.525
avg newton steps: 4.459383753501401
avg pcg iterations per newton step : 898.9343252950231


In [21]:
for time,iters in pcg_iterations:
    print("Time: ",time)
    print("Iters: ",iters)
# print(len(pcg_iterations))

Time:  0
Iters:  454
Time:  0
Iters:  732
Time:  0
Iters:  744
Time:  0
Iters:  411
Time:  0.000909
Iters:  819
Time:  0.000909
Iters:  402
Time:  0.001814
Iters:  881
Time:  0.001814
Iters:  428
Time:  0.002835
Iters:  901
Time:  0.002835
Iters:  939
Time:  0.002835
Iters:  380
Time:  0.003745
Iters:  991
Time:  0.003745
Iters:  942
Time:  0.003745
Iters:  441
Time:  0.004779
Iters:  994
Time:  0.004779
Iters:  987
Time:  0.004779
Iters:  464
Time:  0.005893
Iters:  946
Time:  0.005893
Iters:  934
Time:  0.005893
Iters:  479
Time:  0.006999
Iters:  915
Time:  0.006999
Iters:  915
Time:  0.006999
Iters:  391
Time:  0.007806
Iters:  877
Time:  0.007806
Iters:  877
Time:  0.007806
Iters:  869
Time:  0.007806
Iters:  831
Time:  0.007806
Iters:  478
Time:  0.009053
Iters:  881
Time:  0.009053
Iters:  852
Time:  0.009053
Iters:  801
Time:  0.009053
Iters:  388
Time:  0.009911
Iters:  851
Time:  0.009911
Iters:  812
Time:  0.009911
Iters:  779
Time:  0.009911
Iters:  391
Time:  0.010806
Iter

In [22]:
for time,newt_steps in newton_steps:
    print("time: ",time)
    print("Newt Step: ",newt_steps)
# print(len(newton_steps))

time:  0
Newt Step:  3
time:  0.000909
Newt Step:  1
time:  0.001814
Newt Step:  1
time:  0.002835
Newt Step:  2
time:  0.003745
Newt Step:  2
time:  0.004779
Newt Step:  2
time:  0.005893
Newt Step:  2
time:  0.006999
Newt Step:  2
time:  0.007806
Newt Step:  4
time:  0.009053
Newt Step:  3
time:  0.009911
Newt Step:  3
time:  0.010806
Newt Step:  4
time:  0.011582
Newt Step:  4
time:  0.012512
Newt Step:  3
time:  0.013263
Newt Step:  3
time:  0.01403
Newt Step:  4
time:  0.015078
Newt Step:  4
time:  0.015834
Newt Step:  5
time:  0.016982
Newt Step:  5
time:  0.017971
Newt Step:  5
time:  0.018797
Newt Step:  5
time:  0.019815
Newt Step:  5
time:  0.020731
Newt Step:  5
time:  0.0215
Newt Step:  5
time:  0.022351
Newt Step:  3
time:  0.023154
Newt Step:  5
time:  0.024033
Newt Step:  5
time:  0.025034
Newt Step:  6
time:  0.025794
Newt Step:  7
time:  0.026726
Newt Step:  6
time:  0.027608
Newt Step:  4
time:  0.028365
Newt Step:  5
time:  0.029157
Newt Step:  5
time:  0.030101
Newt

In [35]:
frame_to_time = session.get.log.numbers("frame-to-time")
newton_steps = session.get.log.numbers("newton-steps")
pcg_iterations = session.get.log.numbers("pcg-iter")

frame_times = sorted(frame_to_time)
frame_intervals = [
    (
        f,
        frame_times[i][1],
        frame_times[i + 1][1] if i + 1 < len(frame_times) else float("inf"),
    )
    for i, (f, _) in enumerate(frame_times)
]

def pcg_per_newt(frame_ID):
    frame_id = frame_ID
    start_t, end_t = None, None
    for f, t_start, t_end in frame_intervals:
        if f == frame_id:
            start_t, end_t = t_start, t_end
            break

    if start_t is None or end_t is None:
        print(f"Frame {frame_id}: (not found in frame intervals)")
        return

    newton_in_frame = [(t, n) for t, n in newton_steps if start_t <= t < end_t]
    pcg_in_frame = [n for t, n in pcg_iterations if start_t <= t < end_t]

    print(f"Frame {frame_id}:")
    print(f"  Newton steps: {len(newton_in_frame)}")
    if pcg_in_frame:
        print(f"  Total PCG iterations: {sum(pcg_in_frame)}")
        print(
            f"  Avg PCG iter per Newton step: {sum(pcg_in_frame) / len(newton_in_frame):.2f}")
    else:
        print("  (No PCG iterations logged for this frame)")


for i in range(242):
    pcg_per_newt(i)


Frame 0:
  Newton steps: 18
  Total PCG iterations: 57554
  Avg PCG iter per Newton step: 3197.44
Frame 1:
  Newton steps: 19
  Total PCG iterations: 88359
  Avg PCG iter per Newton step: 4650.47
Frame 2:
  Newton steps: 20
  Total PCG iterations: 96520
  Avg PCG iter per Newton step: 4826.00
Frame 3:
  Newton steps: 19
  Total PCG iterations: 108527
  Avg PCG iter per Newton step: 5711.95
Frame 4:
  Newton steps: 20
  Total PCG iterations: 267391
  Avg PCG iter per Newton step: 13369.55
Frame 5:
  Newton steps: 16
  Total PCG iterations: 234252
  Avg PCG iter per Newton step: 14640.75
Frame 6:
  Newton steps: 21
  Total PCG iterations: 247211
  Avg PCG iter per Newton step: 11771.95
Frame 7:
  Newton steps: 21
  Total PCG iterations: 177344
  Avg PCG iter per Newton step: 8444.95
Frame 8:
  Newton steps: 19
  Total PCG iterations: 85472
  Avg PCG iter per Newton step: 4498.53
Frame 9:
  Newton steps: 3
  Total PCG iterations: 23299
  Avg PCG iter per Newton step: 7766.33
Frame 10:
  N

In [29]:
from collections import defaultdict

# Parse frame time boundaries
frame_to_time = session.get.log.numbers("frame-to-time")
frame_bounds = sorted(frame_to_time)  # (frame, time)

# Invert to time -> frame lookup
time_to_frame = {}
for i in range(len(frame_bounds) - 1):
    t_start = frame_bounds[i][1]
    t_end = frame_bounds[i+1][1]
    for t in [t_start, (t_start + t_end) / 2, t_end]:
        time_to_frame[t] = frame_bounds[i][0]
last_frame = frame_bounds[-1][0]

# Associate Newton steps to frames
newton_steps = session.get.log.numbers("newton-steps")
frame_newton_count = defaultdict(int)

for time, step in newton_steps:
    # Find the most recent frame time <= current Newton step time
    frame = last_frame
    for f, t in reversed(frame_bounds):
        if time >= t:
            frame = f
            break
    frame_newton_count[frame] += 1

# Print result
for frame in sorted(frame_newton_count.keys()):
    print(f"Frame {frame}: {frame_newton_count[frame]} Newton steps")


Frame 0: 18 Newton steps
Frame 1: 19 Newton steps
Frame 2: 20 Newton steps
Frame 3: 19 Newton steps
Frame 4: 20 Newton steps
Frame 5: 16 Newton steps
Frame 6: 21 Newton steps
Frame 7: 21 Newton steps
Frame 8: 19 Newton steps
Frame 9: 3 Newton steps
Frame 10: 4 Newton steps
Frame 11: 1 Newton steps
Frame 12: 3 Newton steps
Frame 13: 3 Newton steps
Frame 14: 3 Newton steps
Frame 15: 1 Newton steps
Frame 16: 4 Newton steps
Frame 17: 1 Newton steps
Frame 18: 4 Newton steps
Frame 19: 3 Newton steps
Frame 20: 1 Newton steps
Frame 21: 3 Newton steps
Frame 22: 3 Newton steps
Frame 23: 3 Newton steps
Frame 24: 1 Newton steps
Frame 25: 4 Newton steps
Frame 26: 1 Newton steps
Frame 27: 3 Newton steps
Frame 28: 3 Newton steps
Frame 29: 2 Newton steps
Frame 30: 2 Newton steps
Frame 31: 2 Newton steps
Frame 32: 3 Newton steps
Frame 33: 2 Newton steps
Frame 34: 3 Newton steps
Frame 35: 2 Newton steps
Frame 36: 4 Newton steps
Frame 37: 3 Newton steps
Frame 38: 2 Newton steps
Frame 39: 3 Newton steps
F

In [20]:
assert "frame-to-time" in logs
frame_to_time = session.get.log.numbers("frame-to-time")

for f,t in frame_to_time:
    print("frame: ",f)
    print("time: ",t)
    for time,newt in newton_steps:
        if time < t:

frame:  0
time:  0
frame:  1
time:  0.01698192418552935
frame:  2
time:  0.03408441902138293
frame:  3
time:  0.05078228155616671
frame:  4
time:  0.06739315699087456
frame:  5
time:  0.08354458282701671
frame:  6
time:  0.10047078516799957
frame:  7
time:  0.11743216484319419
frame:  8
time:  0.133543856034521
frame:  9
time:  0.15068535727914423
frame:  10
time:  0.16704318870324641
frame:  11
time:  0.18368726025801152
frame:  12
time:  0.20004677691031247
frame:  13
time:  0.21812691714148968
frame:  14
time:  0.23502726282458752
frame:  15
time:  0.25113016890827566
frame:  16
time:  0.26926309580449015
frame:  17
time:  0.2836556046968326
frame:  18
time:  0.3022906967671588
frame:  19
time:  0.3170895957155153
frame:  20
time:  0.33446192007977515
frame:  21
time:  0.351908958167769
frame:  22
time:  0.3673977606231347
frame:  23
time:  0.38672591315116733
frame:  24
time:  0.40036637673620135
frame:  25
time:  0.417316930484958
frame:  26
time:  0.43375506170559675
frame:  27
t

In [15]:
for t,n in msec_per_video:
    print("frame: ",t)
    print("time: ",n)

frame:  0
time:  0
frame:  1
time:  4982
frame:  2
time:  8496
frame:  3
time:  8982
frame:  4
time:  10329
frame:  5
time:  23743
frame:  6
time:  25265
frame:  7
time:  20484
frame:  8
time:  21966
frame:  9
time:  11610
frame:  10
time:  4022
frame:  11
time:  3778
frame:  12
time:  3160
frame:  13
time:  3653
frame:  14
time:  2061
frame:  15
time:  1931
frame:  16
time:  2343
frame:  17
time:  1666
frame:  18
time:  2623
frame:  19
time:  2088
frame:  20
time:  2654
frame:  21
time:  4720
frame:  22
time:  2890
frame:  23
time:  3268
frame:  24
time:  2107
frame:  25
time:  2453
frame:  26
time:  2308
frame:  27
time:  2397
frame:  28
time:  2050
frame:  29
time:  3236
frame:  30
time:  2633
frame:  31
time:  2031
frame:  32
time:  1768
frame:  33
time:  2314
frame:  34
time:  2026
frame:  35
time:  2046
frame:  36
time:  2341
frame:  37
time:  2366
frame:  38
time:  4511
frame:  39
time:  4524
frame:  40
time:  3576
frame:  41
time:  3132
frame:  42
time:  3893
frame:  43
time:  

In [ ]:
session.animate()

In [ ]:
session.export.animation()

In [ ]:
# this is for CI
assert session.finished()